### Indentify Candidates
This code filters all the indicators to those having at least 10000 and present the results in a stacked format

2021-09-14: DXG
First pass filtering data down to 60 potential components that have over 10000 observations per predicted variable. Stacked version of data is in dataSetForModelling.csv

2021-09-21: DXG
Modified to pivot stacked data into wide format with no na's
finalDataSetForModelling.csv has the following columns:
CountryName,Year,Life expectancy at birth, total (years), +60 predictors


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('indicators.csv')

In [3]:
df.shape


(5656458, 6)

In [4]:
df.columns

Index(['CountryName', 'CountryCode', 'IndicatorName', 'IndicatorCode', 'Year',
       'Value'],
      dtype='object')

In [5]:
df_groupBy = df.groupby('IndicatorName').size().reset_index(name='counts').sort_values('counts')

In [6]:
df_groupBy.to_csv('IndicatorNameGroupedByCount.csv')

In [7]:
#use fairly arbitrary cutoff at 10000 observations per indicator
cutOff = 10000
df_groupByOver10000 = df_groupBy[df_groupBy['counts'] > cutOff]

In [8]:
df_groupByOver10000.to_csv('IndicatorNameGroupedByCountOver' + str(cutOff) + '.csv')

In [9]:
#filter data to the cutoff limit
df_groupByOver10000.shape

(63, 2)

In [10]:
filteredDataset = pd.merge(left=df,right=df_groupByOver10000)

In [11]:
filteredDataset.to_csv('filteredDataset.csv')

In [12]:
lifeExpectancyTotal = 'Life expectancy at birth, total (years)'
lifeExpectancyMale = 'Life expectancy at birth, male (years)'
lifeExpectancyFemale = 'Life expectancy at birth, female (years)'
dependentVariables = [lifeExpectancyTotal,lifeExpectancyMale,lifeExpectancyFemale]

In [13]:
dependentVariables

['Life expectancy at birth, total (years)',
 'Life expectancy at birth, male (years)',
 'Life expectancy at birth, female (years)']

In [14]:
#now need to split dataset into the indepedent and dependent sets
independentDataRaw = filteredDataset[filteredDataset.IndicatorName.isin(dependentVariables)]

In [15]:
dependentData = filteredDataset[~filteredDataset.IndicatorName.isin(dependentVariables)]

In [16]:
independentDataRawPivoted = pd.pivot(independentDataRaw,values='Value',index=['CountryName','CountryCode','Year'], columns='IndicatorName')

In [17]:
independentDataRawPivotedFlattened = independentDataRawPivoted.reset_index(level=[0,1])

In [18]:
independentDataRawPivotedFlattened.columns.to_flat_index()
independentData = pd.DataFrame(independentDataRawPivotedFlattened.to_records())

In [19]:
independentData
dataSetForModelling =  pd.merge(left=dependentData,right=independentData)

In [20]:
dataSetForModelling.to_csv('datasetForFeatureSelection.csv')

In [21]:
cleanedDataSetForModelling = dataSetForModelling.drop(['IndicatorCode', 'counts'], axis=1)

In [22]:
cleanedDataSetForModelling

,CountryName,CountryCode,IndicatorName,Year,Value,"Life expectancy at birth, female (years)","Life expectancy at birth, male (years)","Life expectancy at birth, total (years)"
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",1960,133.560907,47.883246,45.862949,46.847059
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,1960,87.797601,47.883246,45.862949,46.847059
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",1960,6.634579,47.883246,45.862949,46.847059
3,Arab World,ARB,"Age dependency ratio, young (% of working-age ...",1960,81.023330,47.883246,45.862949,46.847059
4,Arab World,ARB,"Birth rate, crude (per 1,000 people)",1960,47.697888,47.883246,45.862949,46.847059
...,...,...,...,...,...,...,...,...
664250,San Marino,SMR,Arable land (% of land area),2003,16.666667,84.600000,78.000000,81.219512
664251,San Marino,SMR,Arable land (hectares per person),2003,0.035063,84.600000,78.000000,81.219512
664252,San Marino,SMR,Land area (sq. km),2003,60.000000,84.600000,78.000000,81.219512
664253,San Marino,SMR,Population density (people per sq. km of land ...,2003,475.333333,84.600000,78.000000,81.219512


In [23]:
finalDataSetForModelling = cleanedDataSetForModelling.pivot(index=['CountryName','Year','Life expectancy at birth, total (years)'], columns='IndicatorName', values='Value')

In [24]:
finalDataSetForModelling = finalDataSetForModelling.reset_index().dropna()

In [25]:
finalDataSetForModelling = finalDataSetForModelling.reset_index().drop(['index'], axis=1)

# finalDataSetForModelling.csv is the feature set of 60 features, each having at least 10000 observations
This is used for further feature selection to get it down to 10 

In [26]:
finalDataSetForModelling.to_csv("finalDataSetForModelling.csv", index=False)